In [1]:
# get loc of ipython file
import os
notebook_path = os.getcwd()
print(notebook_path)
root = os.path.dirname(os.path.dirname(notebook_path))
print(f"⭐️ root folder for project: {root}")

import sys
sys.path.append(root)

save_root = "Path/root/to/dataset/"

# start with a vocab list
# this related with the gene2vec model
#----> pre-trained part
vocab_loc = root +"/Experiment/support_data/vocab_gene2vec_16906.pkl"
vocab_json = root +"/Experiment/support_data/vocab_16k.json"
target_label = 'Ground_truth' # the label to be predicted
data_path = root + "/Experiment/support_data/cook_toy.h5ad"
print(f"(1) data path: {data_path}")
print(f"(2) vocab loc: {vocab_loc}")
print(f"(3) vocab json: {vocab_json}")
print(f"(4) target label: {target_label}")


/home/pan/Experiments/EXPs/scLLM_workspace/EMT-LM/Experiment/Step_0_preprocess_raw_data
⭐️ root folder for project: /home/pan/Experiments/EXPs/scLLM_workspace/EMT-LM
(1) data path: /home/pan/Experiments/EXPs/scLLM_workspace/EMT-LM/Experiment/support_data/cook_toy.h5ad
(2) vocab loc: /home/pan/Experiments/EXPs/scLLM_workspace/EMT-LM/Experiment/support_data/vocab_gene2vec_16906.pkl
(3) vocab json: /home/pan/Experiments/EXPs/scLLM_workspace/EMT-LM/Experiment/support_data/vocab_16k.json
(4) target label: Ground_truth


In [2]:
# check the data
import scanpy as sc
import pandas as pd
import numpy as np

adata = sc.read(data_path)
print(adata)
print(adata.obs["Ground_truth"].unique())
print(adata.obs["Tissue"].unique())
print(adata.obs["CellLine"].unique())
print(adata.obs["stimulus"].unique())

AnnData object with n_obs × n_vars = 24 × 11058
    obs: 'CellLine', 'stimulus', 'Experiment', 'pseudotimes', 'Ground_truth', 'Tissue', 'batch'
    obsm: 'X_pca', 'X_umap', 'X_umap_pseudo'
['3d', '7d', '1d', '1d_rm', '8h_rm', '3d_rm', '8h', '0d']
Categories (8, object): ['0d', '1d', '1d_rm', '3d', '3d_rm', '7d', '8h', '8h_rm']
['Lung', 'Ovarian', 'Prostate', 'Breast']
Categories (4, object): ['Breast', 'Lung', 'Ovarian', 'Prostate']
['A549', 'OVCA420', 'DU145', 'MCF7']
Categories (4, object): ['A549', 'DU145', 'MCF7', 'OVCA420']
['TGFb']
Categories (1, object): ['TGFb']


In [3]:

from scLLM.Dataset.paras import Dataset_para
# define pre-processing by follow original implementation of scBERT

from scLLM.Dataset.paras import Dataset_para
# define pre-processing by follow original implementation of scBERT
dataset_para_cls = Dataset_para(
                            var_idx=None,
                            obs_idx="Ground_truth",
                            vocab_loc=vocab_json,
                            filter_gene_by_counts=False,
                            filter_cell_by_counts=200,
                            log1p=True,
                            log1p_base=2,

                            #
                            tokenize_name="scBERT",
                            cls_nb=5,
                            data_layer_name="X_log1p",
                            label_key = target_label,#"Ground_truth",#"Ground_truth",

                            test_size=None,#0.2, #use all data to inference
                            binarize=None, # not binarize use original label

                            )

dataset_para_reg = Dataset_para(
        vocab_loc=vocab_json,
        var_idx = None,#"genes.gene_short_name",
        obs_idx="pseudotimes",
        filter_gene_by_counts=False,
        filter_cell_by_counts=200,
        log1p=True,
        log1p_base=2,

        tokenize_name="scBERT",
        cls_nb=1,
        data_layer_name="X_log1p",

        auto_map_str_labels=False,
        label_key = target_label,#"pseudotimes",

        test_size=0.2,#0.2, #use all data to inference
        binarize=None, # not binarize use original label for regression
    )

dataset_para = dataset_para_cls#dataset_para_cls
print(dataset_para)

Dataset_para(var_idx=None, obs_idx='Ground_truth', vocab_loc='/home/pan/Experiments/EXPs/scLLM_workspace/EMT-LM/Experiment/support_data/vocab_16k.json', gene_vocab=None, use_key='X', filter_gene_by_counts=False, filter_cell_by_counts=200, normalize_total=10000.0, result_normed_key='X_normed', log1p=True, result_log1p_key='X_log1p', log1p_base=2, subset_hvg=False, hvg_use_key=None, hvg_flavor='seurat_v3', binning=None, result_binned_key='X_binned', tokenize_name='scBERT', return_pt=True, append_cls=True, include_zero_gene=False, cls_token='<cls>', max_len=16000, pad_token='<pad>', pad_value=-2, cls_appended=True, mask_ratio=0.15, mask_value=-1, preprocessed_loc=None, data_layer_name='X_log1p', label_key='Ground_truth', batch_label_key=None, cls_nb=5, binarize=None, bins=None, bin_min=None, bin_max=None, save_in_obs=True, auto_map_str_labels=True, map_dict=None, n_splits=1, test_size=None, random_state=2023, shuffle=True, sort_seq_batch=False)


In [4]:
# if this is the first time to run, need this block to init translate=True
# init preprocessor
from scLLM.Dataset.Reader import scReader
data_reader = scReader(dataset_para=dataset_para)
# init vocab
data_reader.init_vocab()

# load data
data_reader.load_adata(loc = data_path,translate=True)

## preprocess
data_reader.preprocess()


/home/pan/anaconda3/envs/scLLM/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
scLLM - INFO - Initializing preprocessor ...
scLLM - INFO - use default vocab from dataset_para
scLLM - INFO - load vocab from /home/pan/Experiments/EXPs/scLLM_workspace/EMT-LM/Experiment/support_data/vocab_16k.json
scLLM - INFO - Load data from anndata object.
scLLM - DEBUG - In original adata with gene 11058
scLLM - DEBUG - In original adata with gene 11058
scLLM - DEBUG - processing 0/16906
scLLM - DEBUG - processing 2000/16906
scLLM - DEBUG - processing 4000/16906
scLLM - DEBUG - processing 6000/16906
scLLM - DEBUG - processing 8000/16906
scLLM - DEBUG - processing 10000/16906
scLLM - DEBUG - processing 12000/16906
scLLM - DEBUG - processing 14000/16906
scLLM - DEBUG - processing 16000/16906
scLLM - INFO - create anndata i

In [5]:
target_stimulate = "TGFb"
target_cellline = None #"A549" 'DU145', 'MCF7', 'OVCA420'
map_stimulus = True
map_cancer_type = False

print(f"选择前的数据矩阵形状 {data_reader.adata.shape}")
if map_stimulus:
    stimilus_map = data_reader.adata.obs['stimulus']==target_stimulate
    # filter out cells that are not stimulated by TGFb
    new_adata = data_reader.adata[stimilus_map,:]
    data_reader.adata = new_adata

if map_cancer_type:
    cancer_type_map = data_reader.adata.obs['CellLine']==target_cellline
    # filter out cells that are not stimulated by TGFb
    new_adata = data_reader.adata[cancer_type_map,:]
    data_reader.adata = new_adata

print(f"选择后的数据矩阵形状 {data_reader.adata.shape}")

选择前的数据矩阵形状 (24, 16906)
选择后的数据矩阵形状 (24, 16906)


In [6]:
label_unique = data_reader.adata.obs[dataset_para.label_key].unique()
print(label_unique)

# set label_key manually if needed
#label_dict = {'3d':0, '7d':1, '1d':2, '8h':3, '0d':4 } 
#data_reader.para.map_dict = label_dict

['3d', '7d', '1d', '1d_rm', '8h_rm', '3d_rm', '8h', '0d']
Categories (8, object): ['0d', '1d', '1d_rm', '3d', '3d_rm', '7d', '8h', '8h_rm']


In [7]:
data_reader.adata.obs[dataset_para.label_key]

Mix2_ACGATACCATGTCGAT-1-0         3d
Mix2_GTGCTTCCAATAGCGG-1-0         3d
Mix2_GAGTCCGTCATCTGCC-10-0        3d
Mix2_CATCAGATCTACGAGT-4-1         7d
Mix1_ATGTGTGGTAGCTGCC-4-1         7d
Mix4a_CTCGTCACACTTGGAT-4-1        7d
Mix4b_GAGGTGACATAAAGGT-1-2        1d
Mix2_CCTAAAGGTAGGCTGA-1-2         1d
Mix4b_GCTGCTTTCCACTCCA-10-2       1d
Mix2_ATTCTACAGACGCTTT-4-3      1d_rm
Mix4a_CGTCTACTCTTCGAGA-1-3     1d_rm
Mix4a_TGAGCATGTCCGTCAG-4-3     1d_rm
Mix4a_ATCGAGTTCCACGTGG-1-4     8h_rm
Mix4a_CCAGCGAAGTTAAGTG-4-4     8h_rm
Mix2_CAGTAACCAATCCGAT-4-4      8h_rm
Mix4b_CGATGGCGTGTGTGCC-10-5    3d_rm
Mix2_TGCCCATCACCTATCC-4-5      3d_rm
Mix4a_GCTCCTACAGATGGCA-10-5    3d_rm
Mix3b_CACATAGCAATCGAAA-4-6        8h
Mix2_ACGGGCTCAATGGAGC-7-6         8h
Mix4b_CGGACGTTCCACTGGG-4-6        8h
Mix1_ATCATCTAGTGGCACA-7-7         0d
Mix4b_CGCTATCGTCAGAATA-7-7        0d
Mix1_CTCATTACACACAGAG-7-7         0d
Name: Ground_truth, dtype: category
Categories (8, object): ['0d', '1d', '1d_rm', '3d', '3d_rm', '7d', '8h', '8h

In [8]:
trainset,valset,weights = data_reader.postprocess()

# 输出数据集信息
print("trainset size: ",len(trainset))
print("valset size: ",len(valset)) if valset is not None else print("no valset")
print("weights: ",weights)

scLLM - INFO - Map string labels to int automatically.
scLLM - INFO - Mapping from {'3d': 0, '7d': 1, '1d': 2, '1d_rm': 3, '8h_rm': 4, '3d_rm': 5, '8h': 6, '0d': 7}
/home/pan/Experiments/EXPs/scLLM_workspace/EMT-LM/scLLM/Dataset/Reader.py:263: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  self.adata.obs[key_name]=self.adata.obs[key_name].map(map_dict)
scLLM - INFO - Discritize label Ground_truth in obs_names


trainset size:  24
no valset
weights:  None


In [9]:
import dill
# 为trainset 添加其他labels

dataset_from = "/EMT_Cook_toy_"
#target_task = f"/TrVal_dataset_GT_{target_stimulate}_{target_cellline}.pkl"
target_task = f"TrVal_dataset_{target_label}.pkl"
loc = save_root + dataset_from + target_task
# 保存 trainset 到文件，并关联相应labels
#with open(loc,"wb") as f:
#    dill.dump([trainset,valset,weights,label_dict],f)